## Елизаров Константин, MADE-DS-21

Мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК). <br><br>
**Background:** в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:
- построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
- при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
- а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = './chgk/'

In [3]:
results = pd.read_pickle(DATA_PATH + 'results.pkl')
cups = pd.read_pickle(DATA_PATH + 'tournaments.pkl')
players = pd.read_pickle(DATA_PATH + 'players.pkl')

## 1 Data

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.


Рассматриваются турниры по Что?Где?Когда? за 2019 год (train) и за 2020 год (test)

In [4]:
train_date_cups, test_date_cups = [], []

# Отбираем турниры за 2019 и 2020 год
for cup in cups.values():
    cup_date = pd.to_datetime(cup['dateStart'])
    
    if cup_date.year == 2019:
        train_date_cups.append(cup)
    elif cup_date.year == 2020:
        test_date_cups.append(cup)

train_cups, test_cups = [], []

# Оставляем турниры с известными повопросными результатами и составами команд
for cup in train_date_cups:
    cup_id = cup['id']
    summary = results[cup_id]
    if summary:
        item = summary[0]
        mask = item.get('mask', None)
        if mask is None:
            continue
        team = item.get('teamMembers', [])
        if len(team) == 0:
            continue
        train_cups.append(cup)

for cup in test_date_cups:
    cup_id = cup['id']
    summary = results[cup_id]
    if summary:
        item = summary[0]
        mask = item.get('mask', None)
        if mask is None:
            continue
        team = item.get('teamMembers', [])
        if len(team) == 0:
            continue
        test_cups.append(cup)    


Размеры тренировочной и тестовой выборок:

In [5]:
len(train_cups), len(test_cups)

(674, 173)

## 2 Baseline

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [6]:
import tqdm
from scipy.sparse import hstack
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

Будем использовать модель логистической регрессии, где каждый игрок рассматривается, как независимая единица. Повопросные ответы команды приписываются каждому ее члену и используются в качестве целевых переменных модели (бинарная классификация: ответили на вопрос (1) или нет (0)). В качестве признаков используются ID игроков и вопросов, поэтому веса обученной модели можно рассматривать как силу каждого игрока и сложность каждого вопроса. Данная оценка сил игроков в дальнейшем используется для построения рейтинг-системы игроков.

In [7]:
# Извлечение признаков ID игрока - ID вопроса - Ответил на вопрос (да - 1 / нет - 0)
def feature_extraction(cups):
    ID = 1
    player_ids, target = [], []
    question_ids = []
    for cup in tqdm.tqdm(cups):
        cup_id = cup['id']
        max_len = sum(cup['questionQty'].values()) if cup_id != 6090 else 435
        summary = results[cup_id]

        comp_answers = []
        members = []
        for i, team in enumerate(summary):
            mask = team['mask']
            plrs = [p['player']['id'] for p in team['teamMembers']]
            if not mask or not plrs:
                continue
            
#         Выкидываем из выборки маски со знаком "?"
            if len(mask) < max_len or '?' in mask:
                continue
      
            members.append(plrs)
#         Выкидываем раунды, где вопросы не были засчитаны "Х"
            mask = mask.replace('X', '')
            comp_answers.append(np.array(list(map(int, list(mask)))))
      
        comp_answers = np.array(comp_answers)
        ids_range = np.arange(ID, ID + comp_answers.shape[1])

        for j in range(len(members)):
            for person in members[j]:
                person_vec = (person * np.ones(comp_answers.shape[1])).astype(int)
                for k, item in enumerate(person_vec):
                    player_ids.append(item)
                    question_ids.append(ids_range[k])
                    target.append(comp_answers[j][k])

        ID += comp_answers.shape[1]

    res = pd.DataFrame({'id': np.array(player_ids), 'question_id': np.array(question_ids),
                        'target': np.array(target)})
    return res

In [8]:
train_df = feature_extraction(train_cups)

100%|██████████| 674/674 [00:16<00:00, 41.12it/s] 


In [9]:
train_df.id.unique().shape, train_df.question_id.unique().shape

((57395,), (33233,))

In [10]:
# Используем OneHot - вектора по ID игроков и вопросов в качестве признаков для обучения модели
ohe1 = OneHotEncoder()
vec1 = ohe1.fit_transform(train_df.id.to_numpy().reshape(-1, 1))

ohe2 = OneHotEncoder()
vec2 = ohe2.fit_transform(train_df.question_id.to_numpy().reshape(-1, 1))

X_train = hstack([vec1, vec2])
y_train = train_df[['target']]

In [11]:
X_train.shape, y_train.shape

((17736793, 90628), (17736793, 1))

In [12]:
train_df

,id,question_id,target
0,6212,1,1
1,6212,2,1
2,6212,3,1
3,6212,4,1
4,6212,5,1
...,...,...,...
17736788,210786,33229,0
17736789,210786,33230,0
17736790,210786,33231,0
17736791,210786,33232,0


In [13]:
# Обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train, y_train)

/Users/viconstel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/viconstel/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [14]:
model.coef_.shape

(1, 90628)

In [15]:
# Используем веса модели в качестве рейтинга игроков
rank = pd.DataFrame({'id': ohe1.categories_[0], 'score': model.coef_[0][:len(ohe1.categories_[0])]})
rank = rank.sort_values('score', ascending=False)
rank.head()

,id,score
3861,27403,3.783695
4050,28751,3.679761
604,4270,3.630357
3933,27822,3.605715
4238,30152,3.500463


In [16]:
def id_to_name(person_id):
    info = players[person_id]
    fio = info['surname'], info['name'], info['patronymic']
    return ' '.join(fio)

In [17]:
# Идентификация игроков по их ID
rank['player'] = rank.id.apply(id_to_name)
rank = rank.reindex(columns=['id', 'player', 'score'])
rank.reset_index(inplace=True, drop=True)
rank.head(30)

,id,player,score
0,27403,Руссо Максим Михайлович,3.783695
1,28751,Семушин Иван Николаевич,3.679761
2,4270,Брутер Александра Владимировна,3.630357
3,27822,Савченков Михаил Владимирович,3.605715
4,30152,Сорожкин Артём Сергеевич,3.500463
5,30270,Спешков Сергей Леонидович,3.463432
6,20691,Мереминский Станислав Григорьевич,3.419919
7,18036,Левандовский Михаил Ильич,3.378866
8,87637,Саксонов Антон Владимирович,3.262783
9,26089,Прокофьева Ирина Сергеевна,3.258455


У baseline-модели логистической регрессии получилось довольно неплохое качество, опираясь на текущий рейтинг:
[chgk_rating](https://rating.chgk.info/players.php)

## 3 Spearman and Kendall rank correlation coefficients

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


Для предсказания результатов нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд будем использовать рейтинг команд, рассчитанный как среднее по рейтингам их членов.

В качестве метрики качества на тестовом наборе будем считать ранговые корреляции Спирмена и Кендалла между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров за 2020 год.

In [18]:
from scipy.stats import spearmanr, kendalltau

In [27]:
spearman, kendall = [], []
rank_ids = rank.id.to_numpy()

for cup in test_cups:
    cup_id = cup['id']
    summary = results[cup_id]
    exact_rank, preds_rank = [], []
    if len(summary) < 2:
        continue
        
    for team in summary:
        team_id = team['team']['id']
        exact_rank.append((team_id, team['position']))
        members_ids = [p['player']['id'] for p in team['teamMembers']]
        members_rank = [rank[rank.id == i].score.values[0] if i in rank_ids else 0 for i in members_ids]
        preds_rank.append((team_id, np.mean(members_rank)))
  
    preds_rank = sorted(preds_rank, key=lambda x: x[1], reverse=True)
    preds_places = []
    for i, _ in exact_rank:
        for index, item in enumerate(preds_rank):
            if i == item[0]:
                preds_places.append(index + 1)
                
    exact_rank = [k[1] for k in exact_rank]
    spearman.append(spearmanr(exact_rank, preds_places)[0])
    kendall.append(kendalltau(exact_rank, preds_places)[0])
    
print(f'Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: {np.mean(spearman)}')
print(f'Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: {np.mean(kendall)}')

/Users/viconstel/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/viconstel/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Средний коэффициент ранговой корреляции Спирмена на тестовом множестве: 0.7570849392695089
Средний коэффициент ранговой корреляции Кендалла на тестовом множестве: 0.6017131515489564
